# Introduction
This notebook covers part 3 of the data analysis for the paper: "3D-printed registration phantom for multiscale ultrasound and optical microscopy of small tissue samples."


1.) Rotation: Calculating coordinate system transforms and standard deviations for the fiducial phantom over 5 rotations.

2.) Height: Calculating the coordinate system transforms and standard deviations for the fiducial phantom over 7 heights.

3.) Tendon: Apply the transform onto an image of a tendon.


In [1]:
# Import basic modules
from pathlib import Path
import numpy as np
import SimpleITK as sitk
%matplotlib notebook
import matplotlib.pyplot as plt
import tiffile as tif
import pickle
import os

In [2]:
# These modules all come from the multiscale-imaging package: https://github.com/uw-loci/multiscale
import multiscale.ultrasound.reconstruction as recon
import multiscale.itk.itk_plotting as iplt
import multiscale.itk.registration as reg
import multiscale.itk.process as proc
import multiscale.utility_functions as util

The function definitions below are provided to show the operations applied to acquire the results.  These functions have also been adapted into the multiscale imaging package.  The modified versions are used in the single registration example notebook and the apply registration example notebook.

In [3]:
# These function definitions are used for the analysis.

def open_us(us_path, dynamic_range, spacing, origin):
    """Open the US image, window it to a dynamic range, and rotate it to microscope coordinate axes"""
    raw_image = sitk.ReadImage(str(us_path))
    windowed_image = proc.window_image(raw_image, dynamic_range)
    us_image = rotate_axes_to_microscope(windowed_image)
    us_image.SetSpacing(spacing)
    us_image.SetOrigin(origin)
    us_image.SetDirection([1, 0, 0, 0, 1, 0, 0, 0, -1])
    return us_image

def open_mpm(mpm_path, mpm_origin_path, mpm_spacing):
    """Open the MPM image and set the direction to -1 in Z to mirror microscope convention"""
    positions = positions_from_ometif(mpm_origin_path)
    origin = np.min(positions, 0)
    mpm_image = sitk.ReadImage(str(mpm_path))
    mpm_image.SetSpacing(mpm_spacing)
    mpm_image.SetOrigin(origin)
    mpm_image.SetDirection([1, 0, 0, 0, 1, 0, 0, 0, -1])
    return mpm_image

def rotate_axes_to_microscope(image):
    """Rotate the US axes to be along the microscope axes"""
    arr = sitk.GetArrayFromImage(image)
    arr_rot = np.swapaxes(arr, 0, 1)
    arr_rot = np.flip(arr_rot, 0).astype(np.uint8)
    return sitk.GetImageFromArray(arr_rot)

def positions_from_ometif(file_path):
    """Read a .ome.tif file and grab the image positions as a numpy array"""
    reader = sitk.ImageFileReader()
    reader.SetFileName(str(file_path))
    reader.ReadImageInformation()
    raw_info = reader.GetMetaData('ImageDescription')
    info = tif.xml2dict(raw_info)
    mpm_list = []
    for position in info['OME']['Image']:
        x = position['StageLabel']['X']
        y = position['StageLabel']['Y']
        z = position['Pixels']['Plane'][0]['PositionZ']
        mpm_list.append(np.array([x, y, z]))
    return np.array(mpm_list)

def get_xy_origin(pl_path):
    """Read an ultrasound position list and get the XY origin"""
    raw_pos_list = util.read_json(pl_path)
    pos_list = recon.clean_position_text(raw_pos_list)[0]
    xy_origin = np.min(pos_list, 0)
    return xy_origin

def apply_transform(moving_image, fixed_image, transform_params):
    """Apply a rigid transform based on input parameters"""
    transform = sitk.VersorRigid3DTransform()
    transform.SetParameters(transform_params)
    reg = sitk.Resample(moving_image, fixed_image, transform, sitk.sitkLinear, 0.0, fixed_image.GetPixelID())
    return reg

In [4]:
# The input files are available at https://uwmadison.box.com/s/05ou19dsi2y2wfzgocf1ab7cbbeb4fgn
base_dir = Path(r'F:\Research')

fiducial_dir = Path(base_dir, r'Fiducial paper analysis')
metadata_dir = Path(base_dir, r'Fiducial paper analysis\Metadata images')
mpm_dir = Path(base_dir, r'Fiducial paper analysis\MPM Images\MPM downsampled')
us_dir = Path(base_dir, r'Fiducial paper analysis\Ultrasound')
registered_dir = Path(base_dir, r'Fiducial paper analysis\Registered')

In [5]:
# Assign spacing manually
us_spacing = [25, 25, 25]
mpm_spacing = [8.16, 8.16, 25]

# Specify the dynamic range of the US image
dynamic_range = 50

In [6]:
us_pl_path = Path(fiducial_dir, '2019-05-04_US - 3X 100YSep.pos')
us_xy_origin = get_xy_origin(us_pl_path)

In [7]:
# Open the images
tendon_us_path = Path(us_dir, 'Tendon 2.tif')
tendon_origin = [us_xy_origin[0], us_xy_origin[1], 4]
us_tendon = open_us(tendon_us_path, dynamic_range, us_spacing, tendon_origin)

tendon_mpm_path = Path(mpm_dir, 'Tendon 2.tif')
tendon_origin_path = Path(metadata_dir, 'Tendon 2.ome.tif')
mpm_tendon = open_mpm(tendon_mpm_path, tendon_origin_path, [8.16, 8.16, 8.16])

In [8]:
# Open the transform
coordinate_path = Path(fiducial_dir, 'Coordinate transform.txt')
with open(coordinate_path, 'rb') as fp:
    coordinate_transform = pickle.load(fp)

In [9]:
# Apply the transform US -> MPM
tendon_reg = apply_transform(us_tendon, mpm_tendon, coordinate_transform)
reg_path = Path(registered_dir, 'US Tendon 2.tif')
sitk.WriteImage(tendon_reg, str(reg_path))

In [10]:
# Apply the transform MPM -> US 
tendon_mpm_reg = apply_transform(mpm_tendon, us_tendon, -1*coordinate_transform)
reg_path = Path(registered_dir, 'MPM Tendon 2.tif')
sitk.WriteImage(tendon_mpm_reg, str(reg_path))